Before to begin if you are planning to train a model:

(1) set the runtime with a TPU or GPU 

(2) Make sure a folder tmp_model_dir is not in the working directory from a previous training 

In [1]:
#Print current hardware information (variable in Colab) 

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb  2 08:29:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
#@title
import os
import sys
import tensorflow.compat.v1 as tf

Read from my Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Move to "_modified-EfficientDet" folder and install requirements

In [4]:
os.chdir('drive/MyDrive/_modified-EfficientDet')
sys.path.append('.')
!pip install -r requirements.txt
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-76iagem1
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-76iagem1
     |████████████████████████████████| 6.4 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 82.2 MB/s 
     |████████████████████████████████| 1.1 MB 70.0 MB/s 
     |████████████████████████████████| 120 kB 85.1 MB/s 
     |████████████████████████████████| 213 kB 71.4 MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263954 sha256=81cfa22ee1973f6bfd63b6e4782d3365af1fe7775fe69872c94cbb49e9bee5c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-cxodc27n/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: pycocotools
    Found exi

In [5]:
MODEL = 'efficientdet-d0_1-5'  #@param


# Prepare image and visualization settings.
image_url =  'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'#@param
image_name = 'img.png' #@param
!wget {image_url} -O img.png
import os
img_path = os.path.join(os.getcwd(), 'img.png')

min_score_thresh = 0.35  #@param
max_boxes_to_draw = 200  #@param
line_thickness =   2#@param

import PIL
# Get the largest of height/width and round to 128.
image_size = max(PIL.Image.open(img_path).size)

--2022-02-02 08:30:15--  https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png
Resolving user-images.githubusercontent.com (user-images.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to user-images.githubusercontent.com (user-images.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4080549 (3.9M) [image/png]
Saving to: ‘img.png’

img.png             100%[===================>]   3.89M  --.-KB/s    in 0.09s   

2022-02-02 08:30:15 (41.8 MB/s) - ‘img.png’ saved [4080549/4080549]



Visualize model graph

In [ ]:
!python model_inspect.py --model_name={MODEL} --logdir=logs &> /dev/null
%load_ext tensorboard
%tensorboard --logdir logs

In [15]:
# Get Trash-ICRA19 trainval data (TFrecord previously generated)

file_pattern_train = 'TrashICRA19-train-*-of-19.tfrecord'

images_per_epoch = 1200  #Only the first 4 shards are used as they have annotations for row, plastic and bio only (the other shards add other classes). Each shard has 300 samples.   

print('images_per_epoch = {}'.format(images_per_epoch))

images_per_epoch = 1200


In [ ]:
# Train efficientdet from scratch with backbone checkpoint.
backbone_name = {
    'efficientdet-d0': 'efficientnet-b0',
    'efficientdet-d0_1-5': 'efficientnet-b0', #BiFPN with depth 1, box/class nets with depth 5
    'efficientdet-d0_5-1': 'efficientnet-b0', #BiFPN with depth 5, box/class nets with depth 1
    'efficientdet-d1': 'efficientnet-b1',
    'efficientdet-d1_1-5': 'efficientnet-b1',
    'efficientdet-d2': 'efficientnet-b2',
    'efficientdet-d2_1-5': 'efficientnet-b2',
    'efficientdet-d3': 'efficientnet-b3',
    'efficientdet-d3_1-5': 'efficientnet-b3',
    'efficientdet-d4': 'efficientnet-b4',
    'efficientdet-d4_1-5': 'efficientnet-b4',
    'efficientdet-d5': 'efficientnet-b5',
    'efficientdet-d6': 'efficientnet-b6',
    'efficientdet-d7': 'efficientnet-b6',
}[MODEL]


# generating train tfrecord is large, so we skip the execution here.
import os
if backbone_name not in os.listdir():
  !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckptsaug/{backbone_name}.tar.gz
  !tar xf {backbone_name}.tar.gz

!mkdir tmp_model_dir
# key option: use --backbone_ckpt rather than --ckpt.
# Don't use ema since we only train a few steps.
!python main.py --mode=train_and_eval \
    --train_file_pattern=tfrecord/{file_pattern_train} \
    --val_file_pattern=tfrecord/{file_pattern_train} \
    --model_name={MODEL} \
    --model_dir=tmp_model_dir/{MODEL}-scratch-Trash  \
    --backbone_ckpt={backbone_name} \
    --train_batch_size=4 \
    --eval_batch_size=4 --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=35  \
    --hparams="num_classes=3,moving_average_decay=0,mixed_precision=true"

Visualize the performance and training records

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tmp_model_dir

Evaluate the network latency, i.e. from first convolution to box/class predictions

In [ ]:
print(MODEL)
!python model_inspect.py --runmode=bm --model_name={MODEL} --hparams="mixed_precision=true"

REFERENCES

[1] F. Zocco et al., Towards More Efficient EfficientDets and Low-Light Real-Time Marine Debris Detection, https://arxiv.org/pdf/2203.07155.pdf

[2] https://github.com/google/automl/blob/master/efficientdet/tutorial.ipynb

[3] https://openaccess.thecvf.com/content_CVPR_2020/papers/Tan_EfficientDet_Scalable_and_Efficient_Object_Detection_CVPR_2020_paper.pdf

